In [3]:
%%cmd
pip install --upgrade pip
pip install -r requirements.txt 

Microsoft Windows [Version 10.0.19045.3086]
(c) Microsoft Corporation. All rights reserved.

(venv310) d:\Documents\work\docqa>pip install --upgrade pip
  Using cached pip-23.1.2-py3-none-any.whl (2.1 MB)


ERROR: To modify pip, please run the following command:
D:\Documents\work\ask_api\ask-api\venv310\Scripts\python.exe -m pip install --upgrade pip

[notice] A new release of pip available: 22.3.1 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip



(venv310) d:\Documents\work\docqa>pip install -r requirements.txt 
     -------------------------------------- 396.4/396.4 kB 1.5 MB/s eta 0:00:00
  Using cached Flask-2.3.2-py3-none-any.whl (96 kB)
  Using cached google_search_results-2.4.2.tar.gz (18 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Using cached langchain-0.0.157-py3-none-any.whl (727 kB)
     -------------------------------------- 73.6/73.6 kB 673.5 kB/s eta 0:00:00
     ---------------------------------------- 1.7/1.7 MB 2.3 MB/s eta 0:00:00
     -------------------------------------- 254.8/254.8 kB 1.6 MB/s eta 0:00:00
  Using cached sentencepiece-0.1.97-cp310-cp310-win_amd64.whl (1.1 MB)
  Using cached tiktoken-0.4.0-cp310-cp310-win_amd64.whl (635 kB)
  Using cached async_timeout-4.0.2-py3-none-any.whl (5.8 kB)
  Using cached aiohttp-3.8.4-cp310-cp310-win_amd64.whl (319 kB)
  Using cached openapi_schema_pydantic-1.2.4-py3-none-any.whl (90 kB)
  Using cach

  DEPRECATION: google-search-results is being installed using the legacy 'setup.py install' method, because it does not have a 'pyproject.toml' and the 'wheel' package is not installed. pip 23.1 will enforce this behaviour change. A possible replacement is to enable the '--use-pep517' option. Discussion can be found at https://github.com/pypa/pip/issues/8559


  Running setup.py install for google-search-results: started
  Running setup.py install for google-search-results: finished with status 'done'



[notice] A new release of pip available: 22.3.1 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip



(venv310) d:\Documents\work\docqa>

In [22]:
%jupyter nbextension enable --py widgetsnbextension

Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: ok


check virtual environment

In [1]:
import sys
sys.prefix

'd:\\Documents\\work\\ask_api\\ask-api\\venv310'

In [4]:
import os
import glob
import pprint
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import CharacterTextSplitter
import openai
import chromadb
from chromadb.config import Settings
from chromadb.utils import embedding_functions
from tqdm.notebook import tqdm




openai.api_key = os.getenv("OPENAI_API_KEY")

In [33]:
chromadb_client = chromadb.Client(Settings(
    chroma_db_impl='duckdb+parquet', #chroma_db_impl="duckdb",
    persist_directory="./chroma_db/"
))
sentence_transformer_ef = embedding_functions.SentenceTransformerEmbeddingFunction(model_name="all-MiniLM-L6-v2")
collection_ncc2022 = chromadb_client.get_or_create_collection(name="ncc2022", embedding_function=sentence_transformer_ef) 

# openai_ef = embedding_functions.OpenAIEmbeddingFunction(
#                 api_key=os.environ['OPENAI_API_KEY'],
#                 model_name="text-embedding-ada-002"
#             )

# collection_ncc_openai = client.create_collection(name="ncc2022openai", embedding_function=openai_ef) 

Using embedded DuckDB with persistence: data will be stored in: ./chroma_db/


In [36]:
chromadb_client.list_collections()

No embedding_function provided, using default embedding function: SentenceTransformerEmbeddingFunction


[Collection(name=ncc2022)]

In [22]:
#chromadb_client.delete_collection(name="ncc2022")

In [35]:
file_folder = "D:/Documents/data/national construction code/2022/*.pdf"
for f in glob.glob(file_folder):
    print(f'processing {f}')
    pdf_loader = PyPDFLoader(f)
    docs = pdf_loader.load()
    text_splitter = CharacterTextSplitter(chunk_size=2000, chunk_overlap=0)
    docs = text_splitter.split_documents(docs)
    collection_ncc2022.add(
        documents=[d.page_content for d in docs],
        metadatas=[d.metadata for d in docs],
        ids=[d.metadata['source'] + str(d.metadata['page']) for d in docs]
    )
chromadb_client.persist()

processing D:/Documents/data/national construction code/2022\abcb-housing-provisions-2022-20230501.pdf
processing D:/Documents/data/national construction code/2022\ncc2022-combined-vol2-housing-provisions-20230501.pdf
processing D:/Documents/data/national construction code/2022\ncc2022-consolidated-performance-requirements-20230501.pdf
processing D:/Documents/data/national construction code/2022\ncc2022-volume-one-20230501.pdf
processing D:/Documents/data/national construction code/2022\ncc2022-volume-three-20230501.pdf
processing D:/Documents/data/national construction code/2022\ncc2022-volume-two-20230501.pdf


 impossible to decode XFormObject /Fm0


True

In [43]:
chromadb_client_2 = chromadb.Client(Settings(
    chroma_db_impl="duckdb+parquet",
    persist_directory="D:/Documents/work/docqa/chroma_db"
))
sentence_transformer_ef = embedding_functions.SentenceTransformerEmbeddingFunction(model_name="all-MiniLM-L6-v2")
collection_ncc2022_2 = chromadb_client_2.get_or_create_collection(name="ncc2022", embedding_function=sentence_transformer_ef) 


Using embedded DuckDB with persistence: data will be stored in: D:/Documents/work/docqa/chroma_db


In [44]:
collection_ncc2022_2.query(query_texts=["What's the maximum travel distance between two fire exits in a shopping center?"],
                     include=['documents', 'distances', 'metadatas'],
                     n_results=3)

{'ids': [['D:/Documents/data/national construction code/2022\\ncc2022-volume-one-20230501.pdf185',
   'D:/Documents/data/national construction code/2022\\ncc2022-volume-one-20230501.pdf193',
   'D:/Documents/data/national construction code/2022\\ncc2022-combined-vol2-housing-provisions-20230501.pdf513']],
 'embeddings': None,
 'documents': [['Access and egress\nNCC 2022 Volume One - Building Code of Australia Page 186must be not more than 60 m. \n(6) Assembly buildings — In a Class 9b building other than a school  or early childhood centre , the distance to one of the  \nexits may be 60 m if—  \nthe path of travel from the room concerned to that exit is through another area which is a corridor, hallway, lobby,  (a)\nramp or other circulation space; and  \nthe room is smoke-separated from the circulation space by construction having an FRL of not less than 60/60/60  (b)\nwith every doorway in that construction protected by a tight fitting, self-closing , solid-core door not less than 35

In [42]:
chromadb_client_3 = chromadb.Client(Settings(
    chroma_db_impl="duckdb+parquet",
    persist_directory="D:/Documents/work/docqa/chroma_db"
))
sentence_transformer_ef = embedding_functions.SentenceTransformerEmbeddingFunction(model_name="all-MiniLM-L6-v2")
collection_ncc2022_3 = chromadb_client_3.get_or_create_collection(name="ncc2022", embedding_function=sentence_transformer_ef) 
# file_folder = "D:/Documents/data/national construction code/2022/*.pdf"
# for f in glob.glob(file_folder):
#     print(f'processing {f}')
#     pdf_loader = PyPDFLoader(f)
#     docs = pdf_loader.load()
#     text_splitter = CharacterTextSplitter(chunk_size=2000, chunk_overlap=0)
#     docs = text_splitter.split_documents(docs)
#     collection_ncc2022.add(
#         documents=[d.page_content for d in docs],
#         metadatas=[d.metadata for d in docs],
#         ids=[d.metadata['source'] + str(d.metadata['page']) for d in docs]
#     )
# chromadb_client.persist()
print(chromadb_client_3.list_collections())
text = collection_ncc2022_3.query(query_texts=["What's the maximum travel distance between two fire exits in a shopping center?"],
                     include=['documents', 'distances', 'metadatas'],
                     n_results=3)
text

Using embedded DuckDB with persistence: data will be stored in: D:/Documents/work/docqa/chroma_db
No embedding_function provided, using default embedding function: SentenceTransformerEmbeddingFunction


[Collection(name=ncc2022)]


{'ids': [['D:/Documents/data/national construction code/2022\\ncc2022-volume-one-20230501.pdf185',
   'D:/Documents/data/national construction code/2022\\ncc2022-volume-one-20230501.pdf193',
   'D:/Documents/data/national construction code/2022\\ncc2022-combined-vol2-housing-provisions-20230501.pdf513']],
 'embeddings': None,
 'documents': [['Access and egress\nNCC 2022 Volume One - Building Code of Australia Page 186must be not more than 60 m. \n(6) Assembly buildings — In a Class 9b building other than a school  or early childhood centre , the distance to one of the  \nexits may be 60 m if—  \nthe path of travel from the room concerned to that exit is through another area which is a corridor, hallway, lobby,  (a)\nramp or other circulation space; and  \nthe room is smoke-separated from the circulation space by construction having an FRL of not less than 60/60/60  (b)\nwith every doorway in that construction protected by a tight fitting, self-closing , solid-core door not less than 35

In [37]:
text = collection_ncc2022.query(query_texts=["What's the maximum travel distance between two fire exits in a shopping center?"],
                     include=['documents', 'distances', 'metadatas'],
                     n_results=3)
print(text)

{'ids': [['D:/Documents/data/national construction code/2022\\ncc2022-volume-one-20230501.pdf185', 'D:/Documents/data/national construction code/2022\\ncc2022-volume-one-20230501.pdf193', 'D:/Documents/data/national construction code/2022\\ncc2022-combined-vol2-housing-provisions-20230501.pdf513']], 'embeddings': None, 'documents': [['Access and egress\nNCC 2022 Volume One - Building Code of Australia Page 186must be not more than 60 m. \n(6) Assembly buildings — In a Class 9b building other than a school  or early childhood centre , the distance to one of the  \nexits may be 60 m if—  \nthe path of travel from the room concerned to that exit is through another area which is a corridor, hallway, lobby,  (a)\nramp or other circulation space; and  \nthe room is smoke-separated from the circulation space by construction having an FRL of not less than 60/60/60  (b)\nwith every doorway in that construction protected by a tight fitting, self-closing , solid-core door not less than 35  \nmm t

In [13]:
def build_prompt_with_context(question, context):
    system_prompt = {
        "role": "system",
        "content": """You are a helpful assistant who knows a lot about architecture standards in Australia. 
        I will ask you questions about architecture standards. I will provide a number of relevant paragraphs extracted from 
        the architecture standards for your reference."""
    }
    user_prompt = {
        "role": "user",
        "content": f"""
        Please help me on the question delimited by ``` regarding archituecture standards in Australia. The texts delimited by === are 
        a number of paragraphs extracted from National Construction Code. You can use the texts as reference. Please provide your reply in format of json with 3 fields: "answer",
        "confidence score" between 0 to 1, and "source" which are the original clauses supporting your answer. 
        
        Question:```
        {question}
        ```
        Texts:===
        {os.linesep.join(context)}
        ===
        """
    }
    return [system_prompt,
            user_prompt]





In [8]:
question = "What class of building does a shopping center belong to? What distance should fire staires of shopping center be?"
texts = collection_ncc2022.query(query_texts=[question],
                     include=['documents', 'distances', 'metadatas'],
                     n_results=3
                     )
messages = build_prompt_with_context(question, texts['documents'][0])
response = openai.ChatCompletion.create(
            model='gpt-3.5-turbo',
            messages=messages,
            max_tokens=1000,
        )
response

<OpenAIObject chat.completion id=chatcmpl-7DPEUY9XE2HKBMpyIo47h1aYYB9n9 at 0x17d1034cae0> JSON: {
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "message": {
        "content": "Based on the National Construction Code, a shopping center would fall under the Class 6 category in Australia, as it is a commercial building that serves more than one tenant. Regarding fire stairs, if a Class 6 building has a single storey, a fire-isolated stairway serving the Class 6 should have an automatic smoke detection and alarm system complying with Specification 20. If a Class 6 building has more than one storey, air pressurisation system must be fitted for the fire-isolated exits. The answer confidence is 0.9 as the source clearly mentions that shopping center belongs to Class 6, but it does not explicitly mention the required distance for fire stairs. \n\nAnswer: \n{\n    \"answer\": \"A shopping center belongs to Class 6 in the building classification in Australia. Fire 

In [44]:
texts

{'ids': [['D:/Documents/data/national construction code\\NCC_2019_BCA_Guide.pdf217',
   'D:/Documents/data/national construction code\\NCC_2019_BCA_Guide.pdf93',
   'D:/Documents/data/national construction code\\ncc2022-volume-one-20230501.pdf245']],
 'embeddings': None,
 'documents': [['Services and equipment \nNCC 2019 Guide to BCA Volume One Page 218 Figure E2.2b(1) Plan of shopping centre for Example 1 \nThe specific smoke hazard management provisions for Class 6 buildings containing shops and an enclosed common \nwalkway or mall are similar. Where such a building has a fire compartment with a floor area of more than 2 000 m 2, an \nautomatic smoke exhaust system or an automatic smoke-and-heat vent system to control smoke during a fire is required. \nHowever, where a single storey shop does not open onto the mall and its main public entrance opens to a road or open \nspace, it does not need to have any smoke exhaust or smoke venting, if its floor area does not exceed 2 000 m 2. Thi

In [45]:
question = "What distance should fire staires of shopping center be?"
texts = collection_ncc2022.query(query_texts=[question],
                     include=['documents', 'distances', 'metadatas'],
                     n_results=3
                     )
messages = build_prompt_with_context(question, texts['documents'][0])
response = openai.ChatCompletion.create(
            model='gpt-3.5-turbo',
            messages=messages,
            max_tokens=1000,
        )
response

<OpenAIObject chat.completion id=chatcmpl-7DBmgsxthsbyrBtt5OxfXPUhrz4fb at 0x1909f935120> JSON: {
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "message": {
        "content": "The required distance for fire stairs within a shopping center in Australia is not explicitly stated in the given text. However, the NCC 2019 Guide to BCA Volume One provides information on the width of stairways and ramps where it states that the maximum width of the stairway should be limited to 2 meters, which allows for a reach of 1 meter to either side of the barrier. Wider stairways are allowed but are deemed to add no more than 2 meters of aggregate width. The guide also specifies safe gradients for ramps used as required exits. In case of a shopping center, the required distance for fire stairs would be determined based on the use and size of the building, as well as other factors such as building materials, fire safety systems, among others. Therefore, it is recommended tha

In [33]:
question = "What class of building does a shopping center belong to?"
texts = collection_ncc2022.query(query_texts=[question],
                     include=['documents', 'distances', 'metadatas'],
                     n_results=3
                     )
messages = build_prompt_with_context(question, texts['documents'][0])
response = openai.ChatCompletion.create(
            model='gpt-3.5-turbo',
            messages=messages,
            max_tokens=1000,
        )
response

<OpenAIObject chat.completion id=chatcmpl-7DAg1VCQvHiaA9TdhOTMmWmnjJolO at 0x1909b0c9260> JSON: {
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "message": {
        "content": "Answer: \nA shopping center belongs to Class 6 of the building classification according to the National Construction Code in Australia. Class 6 includes buildings that are for the sale of goods by retail or the supply of services directly to the public, including eating rooms, cafes, restaurants, and service stations. \n\nConfidence Score: 0.95\n\nSource: \n- A3.2 General Provisions of the National Construction Code (NCC) 2015 Building Code of Australia - Volume One. Specifically, Class 6 is defined as \"a shop or other building for the sale of goods by retail or the supply of services direct to the public, including\u2014(a) an eating room, caf\u00e9, restaurant, milk or soft-drink bar; or (b) a dining room, bar area that is not an assembly building, shop or kiosk part of a hotel o

In [46]:
question = "What length of fire stairways of shopping centers should be?"
texts = collection_ncc2022.query(query_texts=[question],
                     include=['documents', 'distances', 'metadatas'],
                     n_results=3
                     )
messages = build_prompt_with_context(question, texts['documents'][0])
response = openai.ChatCompletion.create(
            model='gpt-3.5-turbo',
            messages=messages,
            max_tokens=1000,
        )
response

<OpenAIObject chat.completion id=chatcmpl-7DBnevqFAJMrCky1JuIXWcLgQfxev at 0x1909b0de750> JSON: {
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "message": {
        "content": "Based on the NCC 2019 Guide to BCA Volume One, the required width for stairways is a maximum of 2 meters, effectively permitting a reach of 1 meter to either side. Wider stairways are permitted, with more than 2 meters between the handrails, but are deemed to add no more than 2 meters to the aggregate required width. There are, however, no specific length requirements for fire stairways of shopping centers in Australia mentioned in this text. Therefore, I cannot provide a confident answer to your question.",
        "role": "assistant"
      }
    }
  ],
  "created": 1683377978,
  "id": "chatcmpl-7DBnevqFAJMrCky1JuIXWcLgQfxev",
  "model": "gpt-3.5-turbo-0301",
  "object": "chat.completion",
  "usage": {
    "completion_tokens": 111,
    "prompt_tokens": 1075,
    "total_tokens": 118

In [20]:
question = "What's the maximum travel distance between two fire exits in a shopping center?"
texts = collection_ncc2022.query(query_texts=[question],
                     include=['documents', 'distances', 'metadatas'],
                     n_results=3
                     )
messages = build_prompt_with_context(question, texts['documents'][0])
response = openai.ChatCompletion.create(
            model='gpt-3.5-turbo',
            messages=messages,
            max_tokens=1000,
        )
response

<OpenAIObject chat.completion id=chatcmpl-7EGMHxVMEqZm2OQSdLmlF53P7kgEl at 0x158394acf90> JSON: {
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "message": {
        "content": "Based on the text, the maximum travel distance allowed between two fire exits in a shopping center is 60 meters, as stated in SA D2D6. The maximum distance may only exceed 60 meters if specific requirements are met, such as the room being smoke-separated from the circulation space by a construction with an FRL of not less than 60/60/60, and the maximum distance of travel not exceeding 40 meters within the room and 20 meters from the doorway. The confidence score for this answer is high since the text clearly states the distance between alternative fire exits for Class 2 and 3 buildings and in health-care buildings (45 meters apart) and for all other cases (60 meters apart). \n\n{\n    \"answer\": \"The maximum travel distance allowed between two fire exits in a shopping center is 60

In [48]:
texts

{'ids': [['D:/Documents/data/national construction code\\NCC_2019_BCA_Guide.pdf129',
   'D:/Documents/data/national construction code\\NCC_2019_BCA_Guide.pdf128',
   'D:/Documents/data/national construction code\\NCC_2019_BCA_Guide.pdf136']],
 'embeddings': None,
 'documents': [['Access and egress \nNCC 2019 Guide to BCA Volume One Page 130 blocked. \nUniform distribution of exits— D1.5(a) \nD1.5(a) requires that where multiple exits exist, they must be distributed as uniformly as practicable to improve the level \nof safety when evacuating. \nMinimum and maximum distances are set \nD1.5 specifies the minimum and maximum permitted distances between alternative exits (for example, two exits may be \nlocated next to one another, so long as they are not each other’s alternative in an emergency). Where scissor stairs are \nused, the shafts will be adjacent and separated by fire-resisting construction. However, the access doors to the alternative \nscissor stairs must comply with the minimu

In [24]:
question = "What class building is a shopping center? What's the maximum travel distance between two fire exits in a shopping center?"
texts = collection_ncc2022.query(query_texts=[question],
                     include=['documents', 'distances', 'metadatas'],
                     n_results=3
                     )
messages = build_prompt_with_context(question, texts['documents'][0])
response = openai.ChatCompletion.create(
            model='gpt-3.5-turbo',
            messages=messages,
            max_tokens=1000,
        )
response

IndexError: list index out of range

In [2]:
!pip install panel
!pip install jupyter_bokeh

  Using cached panel-1.0.4-py2.py3-none-any.whl (20.0 MB)
  Using cached bleach-6.0.0-py3-none-any.whl (162 kB)
  Using cached tqdm-4.65.0-py3-none-any.whl (77 kB)
     ---------------------------------------- 10.7/10.7 MB 2.8 MB/s eta 0:00:00
  Using cached xyzservices-2023.5.0-py3-none-any.whl (56 kB)
  Using cached param-1.13.0-py2.py3-none-any.whl (87 kB)
  Using cached pyviz_comms-2.3.0-py2.py3-none-any.whl (43 kB)
  Using cached mdit_py_plugins-0.3.5-py3-none-any.whl (52 kB)
  Using cached Markdown-3.4.3-py3-none-any.whl (93 kB)
     ---------------------------------------- 62.6/62.6 kB 1.6 MB/s eta 0:00:00
  Using cached bokeh-3.1.1-py3-none-any.whl (8.3 MB)
     ---------------------------------------- 87.5/87.5 kB 2.5 MB/s eta 0:00:00
  Using cached linkify_it_py-2.0.2-py3-none-any.whl (19 kB)
  Using cached PyYAML-6.0-cp310-cp310-win_amd64.whl (151 kB)
  Using cached contourpy-1.0.7-cp310-cp310-win_amd64.whl (162 kB)
  Using cached Jinja2-3.1.2-py3-none-any.whl (133 kB)
  Usi


[notice] A new release of pip available: 22.3.1 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


  Using cached jupyter_bokeh-3.0.7-py3-none-any.whl (1.4 MB)
  Using cached ipywidgets-8.0.6-py3-none-any.whl (138 kB)
  Using cached widgetsnbextension-4.0.7-py3-none-any.whl (2.1 MB)
  Using cached jupyterlab_widgets-3.0.7-py3-none-any.whl (198 kB)



[notice] A new release of pip available: 22.3.1 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [9]:
!pip install flask chatterbot

  Using cached Flask-2.3.2-py3-none-any.whl (96 kB)
  Using cached ChatterBot-1.0.5-py2.py3-none-any.whl (67 kB)
  Using cached itsdangerous-2.1.2-py3-none-any.whl (15 kB)
  Using cached blinker-1.6.2-py3-none-any.whl (13 kB)
  Using cached Werkzeug-2.3.4-py3-none-any.whl (242 kB)
  Using cached PyYAML-5.1.2.tar.gz (265 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Using cached pymongo-3.13.0-cp310-cp310-win_amd64.whl (394 kB)
  Using cached Pint-0.21-py3-none-any.whl (286 kB)
     ---------------------------------------- 5.7/5.7 MB 2.6 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Using cached mathparse-0.1.2-py3-none-any.whl (7.2 kB)
  Using cached spacy-2.1.9.tar.gz (30.7 MB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'error'


  error: subprocess-exited-with-error
  
  × pip subprocess to install build dependencies did not run successfully.
  │ exit code: 1
  ╰─> [104 lines of output]
        Using cached setuptools-67.7.2-py3-none-any.whl (1.1 MB)
        Using cached wheel-0.32.3-py2.py3-none-any.whl (21 kB)
        Using cached Cython-0.29.34-py2.py3-none-any.whl (988 kB)
        Using cached cymem-2.0.7-cp310-cp310-win_amd64.whl (29 kB)
        Using cached preshed-2.0.1.tar.gz (113 kB)
        Preparing metadata (setup.py): started
        Preparing metadata (setup.py): finished with status 'done'
        Using cached murmurhash-1.0.9-cp310-cp310-win_amd64.whl (18 kB)
        Using cached thinc-7.0.8.tar.gz (1.9 MB)
        Preparing metadata (setup.py): started
        Preparing metadata (setup.py): finished with status 'done'
        Using cached blis-0.2.4.tar.gz (1.5 MB)
        Preparing metadata (setup.py): started
        Preparing metadata (setup.py): finished with status 'done'
        Using ca

In [7]:



def get_completion(prompt, model="gpt-3.5-turbo"):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0, # this is the degree of randomness of the model's output
    )
    return response.choices[0].message["content"]


def get_completion_from_messages(messages, model="gpt-3.5-turbo", temperature=0):
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=temperature, # this is the degree of randomness of the model's output
    )
#     print(str(response.choices[0].message))
    return response.choices[0].message["content"]


def collect_messages(_):
    prompt = inp.value_input
    inp.value = ''
    context.append({'role':'user', 'content':f"{prompt}"})
    response = get_completion_from_messages(context) 
    context.append({'role':'assistant', 'content':f"{response}"})
    panels.append(
        pn.Row('User:', pn.pane.Markdown(prompt, width=600)))
    panels.append(
        pn.Row('Assistant:', pn.pane.Markdown(response, width=600, style={'background-color': '#F6F6F6'})))
 
    return pn.Column(*panels)

import panel as pn  # GUI
pn.extension()

panels = [] # collect display 

context = [ {'role':'system', 'content':"""
You are OrderBot, an automated service to collect orders for a pizza restaurant. \
You first greet the customer, then collects the order, \
and then asks if it's a pickup or delivery. \
You wait to collect the entire order, then summarize it and check for a final \
time if the customer wants to add anything else. \
If it's a delivery, you ask for an address. \
Finally you collect the payment.\
Make sure to clarify all options, extras and sizes to uniquely \
identify the item from the menu.\
You respond in a short, very conversational friendly style. \
The menu includes \
pepperoni pizza  12.95, 10.00, 7.00 \
cheese pizza   10.95, 9.25, 6.50 \
eggplant pizza   11.95, 9.75, 6.75 \
fries 4.50, 3.50 \
greek salad 7.25 \
Toppings: \
extra cheese 2.00, \
mushrooms 1.50 \
sausage 3.00 \
canadian bacon 3.50 \
AI sauce 1.50 \
peppers 1.00 \
Drinks: \
coke 3.00, 2.00, 1.00 \
sprite 3.00, 2.00, 1.00 \
bottled water 5.00 \
"""} ]  # accumulate messages


inp = pn.widgets.TextInput(value="Hi", placeholder='Enter text here…')
button_conversation = pn.widgets.Button(name="Chat!")

interactive_conversation = pn.bind(collect_messages, button_conversation)

dashboard = pn.Column(
    inp,
    pn.Row(button_conversation),
    pn.panel(interactive_conversation, loading_indicator=True, height=300),
)

dashboard

ImportError: cannot import name 'warn' from 'bokeh.util.warnings' (d:\Documents\work\langchain-study\python310\lib\site-packages\bokeh\util\warnings.py)

Column
    [0] TextInput(placeholder='Enter text here…')
    [1] Row
        [0] Button(name='Chat!')
    [2] ParamFunction(function, _pane=Column, height=300, loading_indicator=True)

In [3]:
def calculate_score(num, central_pile, memo):
    if num == 1 or num not in central_pile:
        return 0

    if num in memo:
        return memo[num]

    max_score = 0
    for factor in range(2, int(num/2) + 1):
        if num % factor == 0 and factor in central_pile:
            new_central_pile = central_pile.copy()
            new_central_pile.remove(factor)
            max_score = max(max_score, num + calculate_score(factor, new_central_pile, memo))

    memo[num] = max_score
    return max_score


central_pile = list(range(1, 21))
memo = {}

max_score = 0
for num in central_pile:
    max_score = max(max_score, calculate_score(num, central_pile, memo))

print("Maximum Score:", max_score)


Maximum Score: 20


In [6]:
system_prompt = {
        "role": "system",
        "content": """You are a mathmatician."""
    }
user_prompt = {
        "role": "user",
        "content": f"""
        The list of GPFs (Greatest Primary Factor) of successive composites is called a GFP sequence. For example, the sequence of GPFs for the composites from 40 to  49 is 5, 7, 11, 5, 23, 3, 7. Note that 41, 43 and 47 are prime and do not contribute to the sequence of GPFs. 
Based on this information, please explain why successive composites that give the sequence of GPFs 41, 19, 79 must all have at least four digits.
        """
    }
messages = [system_prompt, user_prompt]
response = openai.ChatCompletion.create(
            model='gpt-4',
            messages=messages,
            max_tokens=1000,
        )
response

<OpenAIObject chat.completion id=chatcmpl-7IxWqGGp40OfixYfQDHJub0Fvno94 at 0x250dd976d40> JSON: {
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "message": {
        "content": "To better understand the reasoning behind this, let's break down the problem step by step:\n\n1. We know that the GPFs of the successive composite numbers in question are 41, 19, and 79.\n\n2. First, let's consider the GPF of 41. Since 41 is a prime number, the smallest composite number with a GPF of 41, will have the form (41 * a), where a is the smallest integer greater than 1. Hence, the smallest composite with a GPF of 41 is 41 * 2 = 82. But this is a three-digit number, so we need to look further.\n\n3. Next, let's consider the GPF of 19. As we know that the GPF of the previous composite number was 41, then this composite number will have the form (19 * b), where b is an integer, such that 41 divides (19 * b - 1). The smallest integer b that satisfies this condition is 9, becau

In [1]:
import panel as pn  # GUI
pn.extension()

def collect_messages(debug=False):
    user_input = inp.value_input

    panels.append(
        pn.Row("User:", pn.pane.Markdown(user_input, width=600))
    )
    panels.append(
        pn.Row("Chatbot:", pn.pane.Markdown(f"I hear '{user_input}'", width=600, styles={'background-color': '#F6F6F6'}))
    )
    return pn.Column(*panels)

panels = [] # collect display 
inp = pn.widgets.TextInput(placeholder='Enter text here…')
button_conversation = pn.widgets.Button(name="Service Assistant")

interactive_conversation = pn.bind(collect_messages, button_conversation)
dashboard = pn.Column(
    inp,
    pn.Row(button_conversation),
    pn.panel(interactive_conversation, loading_indicator=True, height=300)
)
dashboard

BokehModel(combine_events=True, render_bundle={'docs_json': {'a8183637-a835-40ef-a542-8c7bfec12c9c': {'version…

C:\Users\ellen\AppData\Local\Temp\ipykernel_2908\3404714276.py:8: PanelDeprecationWarning: 'style' is deprecated and will be removed in version 1.1, use 'styles' instead.
  pn.Row("Chatbot:", pn.pane.Markdown(f"I hear '{user_input}'", width=600, style={'background-color': '#F6F6F6'}))


BokehModel(combine_events=True, render_bundle={'docs_json': {'e41b01f8-2408-40bb-a6e2-ae794cba14d8': {'version…